In [2]:
pwd

'd:\\Data\\IIIT_Delhi\\2_Semester\\1_IR\\Assignments\\A2'

In [3]:
#Import header files
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import PorterStemmer
from num2words import num2words
import pickle
import os
import re
import numpy as np 
# np.set_printoptions(threshold = np.inf)
import pandas as pd
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shrad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#Retrieve all folders
locations = [pos[0] for pos in os.walk(str(os.getcwd())+'/'+'stories'+'/')]
len_locations = len(locations[0])
locations[0] = locations[0][:len_locations-1]

print("Folder Locations = ",locations)

Folder Locations =  ['d:\\Data\\IIIT_Delhi\\2_Semester\\1_IR\\Assignments\\A2/stories', 'd:\\Data\\IIIT_Delhi\\2_Semester\\1_IR\\Assignments\\A2/stories/FARNON', 'd:\\Data\\IIIT_Delhi\\2_Semester\\1_IR\\Assignments\\A2/stories/SRE']


In [5]:
#Load all files from index file
dataset = []

flag = False

for i in locations:
    file = open(i + "/index.html", 'r')
    text = file.read().strip()
    file.close()

    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)

    if flag == False:
        file_name = file_name[2:]
        flag = True

    for j in range(len(file_name)):
        dataset.append((str(i) + "/" + str(file_name[j]), file_title[j]))

N = len (dataset)
print("Total number of files =",N)

Total number of files = 467


In [6]:
#Preprocessing functions
def l_case(data):
    return np.char.lower(data)

def punctuation(data):
    symbols = "!\"#$%&()*+-./:;,<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    return data

def apostrophe(data):
    return np.char.replace(data, "'", "")

def stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_word = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_word = new_word + " " + w
    return np.char.strip(new_word)

def lemming(data):
    wordnet_lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(str(data))
    new_word = ""
    for w in tokens:
        new_word = new_word + " " + wordnet_lemmatizer.lemmatize(w)
    return np.char.strip(new_word)

def stem_text(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def process_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text


In [8]:
def preprocess(text):
    text = l_case(text)
    text = punctuation(text)
    text = apostrophe(text)
    text = stop_words(text)
    # text = process_numbers(text)
    # text = lemming(text)
    # text = stem_text(text)
    # text = punctuation(text)
    # text = process_numbers(text)
    # text = stem_text(text) 
    # text = punctuation(text) 
    # text = stop_words(text)

    return text

In [9]:
processed_text = []
processed_title = []

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

In [10]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [11]:
# len(processed_text[466])
len(processed_text)

467

In [12]:
total_vocab_size = len(DF)

In [13]:
total_vocab_size

52385

In [14]:
total_vocab = [x for x in DF]

In [15]:
print(total_vocab[:20])

['shareware', 'trial', 'project', 'freeware', 'need', 'support', 'continue', '100', 'west', '53', 'north', 'jim', 'prentice', 'copyright', '1990', 'brandon', 'manitoba', 'canada', 'magic', 'phrase']


In [16]:

def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [17]:
# Term Frequency
# doc = 0
TF_idf = np.zeros((N, len(total_vocab)))

# tf_idf = {}
tf_idf1 =[]

for i in range(N):
    tf_vec = []
    tf_vec1 =[]
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    # for token in np.unique(tokens):
        
    #     tf = counter[token]/words_count
    #     df = doc_freq(token)
    #     idf = np.log((N+1)/(df+1))
        
    #     tf_idf[doc, token] = tf*idf

    for token in total_vocab:
        if token in processed_text[i]:
           
            tf = counter[token]/words_count
            tf_vec.append(tf)
            # df = doc_freq(token)
            # idf = np.log((N+1)/(df+1))
            # tf_idf[doc, token] = tf*idf
           
        else:
            
            tf = 0
            tf_vec.append(tf)
            # df = doc_freq(token)
            # idf = np.log((N+1)/(df+1))
            # tf_idf[doc, token] = tf*idf

    for j in range(len(total_vocab)):

         df = doc_freq(total_vocab[j])
         idf = np.log((N+1)/(df+1))

    #     # if(total_vocab[j] in processed_text[i][j]  ):
         temp = tf_vec[j]*idf
         tf_vec1.append(temp)
   
         TF_idf[i][j] = tf_vec[j]*idf
    tf_idf1.append(tf_vec1)


 
    # doc += 1

np.save( "TFidftermfrequency",TF_idf)


KeyboardInterrupt: 

In [56]:
def matching_score_2(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)

    Q = np.zeros((len(total_vocab)))
    q_vec =[]
    tfscore = 0
    alltfscore =[]

    for i in range(len(total_vocab)):
        if total_vocab[i] in tokens:
            Q[i] = 1
        else:
            Q[i] = 0
    # print(Q)

    for token in total_vocab:
        if token in tokens:
            q_vec.append(1)
        else:
            q_vec.append(0)
    # print(q_vec)

    for j in range(N):
        tfscore = 0
        for k in range(len(total_vocab)):
            if(q_vec[k] == 1):
                tfscore += TF_idf[j][k]
            else:
                tfscore = tfscore
        alltfscore.append(tfscore)
    # print(alltfscore)

    alltfscore_sorted = sorted(alltfscore, reverse=True)
    # print(alltfscore_sorted)

    # print(sorted(range(len(alltfscore)), key=lambda i: alltfscore[i], reverse=True)[:5])
    print("")
    docName = []

    li=[]
    for i in range(len(alltfscore)):
      li.append([alltfscore[i],i])
    li.sort( reverse=True)
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    print((sort_index)[:5])
    for index in((sort_index)[:5]):
        head, tail = os.path.split(dataset[index][0])
        docName.append(tail)
        print(alltfscore[index])
    print(docName,"\n")
    # 11, 129, 224, 236, 366
    # print(alltfscore[11], alltfscore[129], alltfscore[224], alltfscore[236], alltfscore[366]) 
    
    # Document Vectorization
    cosine_score_term_freq = []
    from sklearn.metrics.pairwise import cosine_similarity
    D = np.zeros((N, total_vocab_size))
    q_vec_1 = np.asarray(q_vec, dtype=np.float32)
    for i in TF_idf:       
        doc = i.reshape(1,total_vocab_size)
        doc = np.array(doc)
        # print(type(doc))
        query = q_vec_1.reshape(1,total_vocab_size)
        query = np.array(query)
        cosSim = cosine_similarity(doc, query)
        # list1 = cosSim[0].tolist()
        list1 = cosSim[0].item()
        cosine_score_term_freq.append(list1)
    print(cosine_score_term_freq)

    cosine_score_term_freq_sorted = sorted(cosine_score_term_freq)
    docName = []
    top5 = cosine_score_term_freq_sorted[-5:]
    print(top5)
    for i in range(5):        
        print(top5[4-i]," and index is ",cosine_score_term_freq.index(top5[4-i]))
        index = cosine_score_term_freq.index(top5[4-i])
        head, tail = os.path.split(dataset[index][0])
        tail.strip()
        docName.append(tail.strip())
        docName.append(" with Cosine Similarity Value:")
        docName.append(top5[4-i])
        docName.append("\n")
    print("\nList of Top Documents wrt Cosine Similarity for i/p query is:")
    print(*docName)
    
matching_score_2(5, "THE CRAB AND THE HERON")  

Matching Score

Query: THE CRAB AND THE HERON

['crab', 'heron']

[122, 26, 270, 418, 169]
0.41362530209306514
0.0016366746585185015
0.001309303929884175
0.0008390118423461789
0.000632598929387192
['crabhern.txt', 'aesop11.txt', 'long1-3.txt', 'timem.hac', 'fgoose.txt'] 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018439642087588456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7973139054443426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [57]:
# Binary tf-idf

import pandas as pd

TF_idf0 =  np.zeros((N, len(total_vocab)))
tf_idf0 = []
Bin_tf = []
bin_tfidf_df = pd.DataFrame()
for p in range(N):

    sent_vec = []
    tf_vec0 =[]

    for token in total_vocab:
        if token in processed_text[p]:
            sent_vec.append(1)
        else:
            sent_vec.append(0)   

    

    for j in range(len(total_vocab)):

        df0 = doc_freq(total_vocab[j])
        idf0 = np.log((N+1)/(df0+1))

        temp = sent_vec[j]*idf0
        tf_vec0.append(temp)
       
        TF_idf0[p][j]= sent_vec[j]*idf0

    Bin_tf.append(sent_vec)
    tf_idf0.append(tf_vec0)
    # bin_tfidf_df.append(tf_vec0)
    # bin_tfidf_df = bin_tfidf_df.append(tf_vec0,ignore_index=True)
    bin_tfidf_df[i] = tf_vec0

  

In [58]:
np.save( "TFidfBinary",TF_idf0)

In [59]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)

    Q = np.zeros((len(total_vocab)))
    q_vec =[]
    tfscore = 0
    alltfscore =[]

    for i in range(len(total_vocab)):
        if total_vocab[i] in tokens:
            Q[i] = 1
        else:
            Q[i] = 0
    # print(Q)

    for token in total_vocab:
        if token in tokens:
            q_vec.append(1)
        else:
            q_vec.append(0)
    # print(q_vec)

    for j in range(N):
        tfscore = 0
        for k in range(len(total_vocab)):
            if(q_vec[k] == 1):
                tfscore += TF_idf0[j][k]
            else:
                tfscore = tfscore
        alltfscore.append(tfscore)
    # print(alltfscore)

    alltfscore_sorted = sorted(alltfscore, reverse=True)
    # print(alltfscore_sorted)

    # print(sorted(range(len(alltfscore)), key=lambda i: alltfscore[i], reverse=True)[:5])
    print("")
    docName = []

    li=[]
    for i in range(len(alltfscore)):
      li.append([alltfscore[i],i])
    li.sort( reverse=True)
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    print((sort_index)[:5])
    for index in((sort_index)[:5]):
        head, tail = os.path.split(dataset[index][0])
        docName.append(tail)
        print(alltfscore[index])
    print(docName)
    # 11, 129, 224, 236, 366
    # print(alltfscore[11], alltfscore[129], alltfscore[224], alltfscore[236], alltfscore[366])  

    # Document Vectorization
    cosine_score_binary = []
    from sklearn.metrics.pairwise import cosine_similarity
    D = np.zeros((N, total_vocab_size))
    q_vec_1 = np.asarray(q_vec, dtype=np.float32)
    for i in TF_idf0:       
        doc = i.reshape(1,total_vocab_size)
        doc = np.array(doc)
        # print(type(doc))
        query = q_vec_1.reshape(1,total_vocab_size)
        query = np.array(query)
        cosSim = cosine_similarity(doc, query)
        # list1 = cosSim[0].tolist()
        list1 = cosSim[0].item()
        cosine_score_binary.append(list1)
    print(cosine_score_binary)

    cosine_score_binary_sorted = sorted(cosine_score_binary)
    docName = []
    top5 = cosine_score_binary_sorted[-5:]
    print(top5)
    for i in range(5):        
        print(top5[4-i]," and index is ",cosine_score_binary.index(top5[4-i]))
        index = cosine_score_binary.index(top5[4-i])
        head, tail = os.path.split(dataset[index][0])
        tail.strip()
        docName.append(tail.strip())
        docName.append(" with Cosine Similarity Value:")
        docName.append(top5[4-i])
        docName.append("\n")
    print("\nList of Top Documents wrt Cosine Similarity for i/p query(Binary TF Weighting) is:")
    print(*docName)  
    
matching_score(5, "THE CRAB AND THE HERON")    
# matching_score(5, "I will endeavour, in my statement, to avoid such terms as would serve to limit the events to any particular place, or give a clue as to the people concerned.")
# matching_score(5, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Matching Score

Query: THE CRAB AND THE HERON

['crab', 'heron']

[122, 26, 418, 270, 169]
9.406564833939129
9.406564833939129
4.356708826689592
4.356708826689592
4.356708826689592
['crabhern.txt', 'aesop11.txt', 'timem.hac', 'long1-3.txt', 'fgoose.txt']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.025439293354229037, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2401234519815826, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [60]:
# TF-IDF RAW COUNT
# doc = 0
TF_idf_raw = np.zeros((N, len(total_vocab)))

# tf_idf_r = {}
tf_idf1_r =[]

for i in range(N):
    tf_vec_r = []
    tf_vec1_r =[]
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    

    for token in total_vocab:
        if token in processed_text[i]:

            tf = counter[token]
            # tf = counter[token]/words_count
            tf_vec_r.append(tf)
          
           
        else:
            
            tf = 0
            tf_vec_r.append(tf)
          

    for j in range(len(total_vocab)):

         df = doc_freq(total_vocab[j])
         idf = np.log((N+1)/(df+1))

    
         temp = tf_vec_r[j]*idf
         tf_vec1_r.append(temp)
   
         TF_idf_raw[i][j] = tf_vec_r[j]*idf
    tf_idf1_r.append(tf_vec1_r)


 
    # doc += 1

np.save( "TFidfrawcount",TF_idf_raw)


In [61]:
def matching_score_3(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)

    Q = np.zeros((len(total_vocab)))
    q_vec =[]
    tfscore = 0
    alltfscore =[]

    for i in range(len(total_vocab)):
        if total_vocab[i] in tokens:
            Q[i] = 1
        else:
            Q[i] = 0
    # print(Q)

    for token in total_vocab:
        if token in tokens:
            q_vec.append(1)
        else:
            q_vec.append(0)
    # print(q_vec)

    for j in range(N):
        tfscore = 0
        for k in range(len(total_vocab)):
            if(q_vec[k] == 1):
                tfscore += TF_idf_raw[j][k]
            else:
                tfscore = tfscore
        alltfscore.append(tfscore)
    # print(alltfscore)

    alltfscore_sorted = sorted(alltfscore, reverse=True)
    # print(alltfscore_sorted)

    # print(sorted(range(len(alltfscore)), key=lambda i: alltfscore[i], reverse=True)[:5])
    print("")
    docName = []

    li=[]
    for i in range(len(alltfscore)):
      li.append([alltfscore[i],i])
    li.sort( reverse=True)
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    print((sort_index)[:5])
    for index in((sort_index)[:5]):
        head, tail = os.path.split(dataset[index][0])
        docName.append(tail)
        print(alltfscore[index])
    print(docName)
    # 11, 129, 224, 236, 366
    # print(alltfscore[11], alltfscore[129], alltfscore[224], alltfscore[236], alltfscore[366])    
     # Document Vectorization
    cosine_score_raw = []
    from sklearn.metrics.pairwise import cosine_similarity
    D = np.zeros((N, total_vocab_size))
    q_vec_1 = np.asarray(q_vec, dtype=np.float32)
    for i in TF_idf_raw:       
        doc = i.reshape(1,total_vocab_size)
        doc = np.array(doc)
        # print(type(doc))
        query = q_vec_1.reshape(1,total_vocab_size)
        query = np.array(query)
        cosSim = cosine_similarity(doc, query)
        # list1 = cosSim[0].tolist()
        list1 = cosSim[0].item()
        cosine_score_raw.append(list1)
    print(cosine_score_raw)

    cosine_score_raw_sorted = sorted(cosine_score_raw)
    docName = []
    top5 = cosine_score_raw_sorted[-5:]
    print(top5)
    for i in range(5):        
        print(top5[4-i]," and index is ",cosine_score_raw.index(top5[4-i]))
        index = cosine_score_raw.index(top5[4-i])
        head, tail = os.path.split(dataset[index][0])
        tail.strip()
        docName.append(tail.strip())
        docName.append(" with Cosine Similarity Value:")
        docName.append(top5[4-i])
        docName.append("\n")
    print("\nList of Top Documents wrt Cosine Similarity for i/p query(Raw count) is:")
    print(*docName)  
    
matching_score_3(5, "THE CRAB AND THE HERON") 

Matching Score

Query: THE CRAB AND THE HERON

['crab', 'heron']

[122, 26, 418, 270, 169]
83.96593632489221
31.19010896738709
13.070126480068776
8.713417653379183
4.356708826689592
['crabhern.txt', 'aesop11.txt', 'timem.hac', 'long1-3.txt', 'fgoose.txt']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018439642087588422, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7973139054443423, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [18]:
# Log normalisation 

# doc = 0
TF_idf_logn = np.zeros((N, len(total_vocab)))

# tf_idf_ln = {}
tf_idf1_ln =[]

for i in range(N):
    tf_vec = []
    tf_vec1 =[]
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    

    for token in total_vocab:
        if token in processed_text[i]:
           
            # tf = counter[token]/words_count
            tf = np.log(1 + counter[token])
            tf_vec.append(tf)
          
        else:
            
            tf = 0
            tf_vec.append(tf)
           

    for j in range(len(total_vocab)):

         df = doc_freq(total_vocab[j])
         idf = np.log((N+1)/(df+1))

    
         temp = tf_vec[j]*idf
         tf_vec1.append(temp)
   
         TF_idf_logn[i][j] = tf_vec[j]*idf
    tf_idf1_ln.append(tf_vec1)


 
    # doc += 1

np.save( "TFidflogNormalisation",TF_idf_logn)


In [20]:
def matching_score_4(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)

    Q = np.zeros((len(total_vocab)))
    q_vec =[]
    tfscore = 0
    alltfscore =[]

    for i in range(len(total_vocab)):
        if total_vocab[i] in tokens:
            Q[i] = 1
        else:
            Q[i] = 0
    # print(Q)

    for token in total_vocab:
        if token in tokens:
            q_vec.append(1)
        else:
            q_vec.append(0)
    # print(q_vec)

    for j in range(N):
        tfscore = 0
        for k in range(len(total_vocab)):
            if(q_vec[k] == 1):
                tfscore += TF_idf_logn[j][k]
            else:
                tfscore = tfscore
        alltfscore.append(tfscore)
    # print(alltfscore)

    alltfscore_sorted = sorted(alltfscore, reverse=True)
    # print(alltfscore_sorted)

    # print(sorted(range(len(alltfscore)), key=lambda i: alltfscore[i], reverse=True)[:5])
    print("")
    docName = []

    li=[]
    for i in range(len(alltfscore)):
      li.append([alltfscore[i],i])
    li.sort( reverse=True)
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    print((sort_index)[:5])
    for index in((sort_index)[:5]):
        head, tail = os.path.split(dataset[index][0])
        docName.append(tail)
        print(alltfscore[index])
    print(docName)
    # 11, 129, 224, 236, 366
    # print(alltfscore[11], alltfscore[129], alltfscore[224], alltfscore[236], alltfscore[366])    

    # Document Vectorization
    cosine_score_log_nrml = []
    from sklearn.metrics.pairwise import cosine_similarity
    D = np.zeros((N, total_vocab_size))
    q_vec_1 = np.asarray(q_vec, dtype=np.float32)
    for i in TF_idf_logn:       
        doc = i.reshape(1,total_vocab_size)
        doc = np.array(doc)
        # print(type(doc))
        query = q_vec_1.reshape(1,total_vocab_size)
        query = np.array(query)
        cosSim = cosine_similarity(doc, query)
        # list1 = cosSim[0].tolist()
        list1 = cosSim[0].item()
        cosine_score_log_nrml.append(list1)
    print(cosine_score_log_nrml)

    cosine_score_log_nrml_sorted = sorted(cosine_score_raw)
    docName = []
    top5 = cosine_score_log_nrml_sorted[-5:]
    print(top5)
    for i in range(5):        
        print(top5[4-i]," and index is ",cosine_score_log_nrml.index(top5[4-i]))
        index = cosine_score_log_nrml.index(top5[4-i])
        head, tail = os.path.split(dataset[index][0])
        tail.strip()
        docName.append(tail.strip())
        docName.append(" with Cosine Similarity Value:")
        docName.append(top5[4-i])
        docName.append("\n")
    print("\nList of Top Documents wrt Cosine Similarity for i/p query(Log Normalization) is:")
    print(*docName)  
# matching_score_5(5, "THE CRAB AND THE HERON")
    
matching_score_4(5, "THE CRAB AND THE HERON") 

Matching Score

Query: THE CRAB AND THE HERON

['crab', 'heron']

[122, 26, 418, 270, 169]
21.54259923161554
11.978057375992861
6.039680879481035
4.786333855150008
3.0198404397405176
['crabhern.txt', 'aesop11.txt', 'timem.hac', 'long1-3.txt', 'fgoose.txt']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03016404874411095, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5607367318281735, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [21]:
# Double  Normalisation 

TF_idf_dbl = np.zeros((N, len(total_vocab)))

# tf_idf = {}
tf_idf1_dbl =[]
max_count = 0

# for i in range(N):
#     all_word_count =[]
#     tokens = processed_text[i]
#     words_count = len(tokens + processed_title[i])
#     all_word_count.append(words_count)
# max_count = max(all_word_count)



for i in range(N):
    tf_vec = []
    tf_vec1 =[]
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    # words_count = len(tokens + processed_title[i])
    
    count_max_freq = counter.most_common(1)[0][1]
    # print(count_max_freq)

    for token in total_vocab:
        if token in processed_text[i]:
           
            # tf = 0.5+0.5*(counter[token]/max_count)
            tf = 0.5+0.5*(counter[token]/count_max_freq)
            # 0.5+0.5*(f(t,d)/ max(f(t`,d))
           
            tf_vec.append(tf)
          
           
        else:
            
            tf = 0
            tf_vec.append(tf)
          

    for j in range(len(total_vocab)):

         df = doc_freq(total_vocab[j])
         idf = np.log((N+1)/(df+1))

         temp = tf_vec[j]*idf
         tf_vec1.append(temp)
   
         TF_idf_dbl[i][j] = tf_vec[j]*idf
    tf_idf1_dbl.append(tf_vec1)


 
    # doc += 1

np.save( "TFidfdoublenormalisation",TF_idf_dbl)

In [22]:
def matching_score_5(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)

    Q = np.zeros((len(total_vocab)))
    q_vec =[]
    tfscore = 0
    alltfscore =[]

    for i in range(len(total_vocab)):
        if total_vocab[i] in tokens:
            Q[i] = 1
        else:
            Q[i] = 0
    # print(Q)

    for token in total_vocab:
        if token in tokens:
            q_vec.append(1)
        else:
            q_vec.append(0)
    # print(q_vec)

    for j in range(N):
        tfscore = 0
        for k in range(len(total_vocab)):
            if(q_vec[k] == 1):
                tfscore += TF_idf_dbl[j][k]
            else:
                tfscore = tfscore
        alltfscore.append(tfscore)
    # print(alltfscore)

    alltfscore_sorted = sorted(alltfscore, reverse=True)
    # print(alltfscore_sorted)

    # print(sorted(range(len(alltfscore)), key=lambda i: alltfscore[i], reverse=True)[:5])
    print("")
    docName = []

    li=[]
    for i in range(len(alltfscore)):
      li.append([alltfscore[i],i])
    li.sort( reverse=True)
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    print((sort_index)[:5])
    for index in((sort_index)[:5]):
        head, tail = os.path.split(dataset[index][0])
        docName.append(tail)
        print(alltfscore[index])
    print(docName)
    # 11, 129, 224, 236, 366
    # print(alltfscore[11], alltfscore[129], alltfscore[224], alltfscore[236], alltfscore[366])    

    # Document Vectorization
    cosine_score_dbl_nrml = []
    from sklearn.metrics.pairwise import cosine_similarity
    D = np.zeros((N, total_vocab_size))
    q_vec_1 = np.asarray(q_vec, dtype=np.float32)
    for i in TF_idf_dbl:       
        doc = i.reshape(1,total_vocab_size)
        doc = np.array(doc)
        # print(type(doc))
        query = q_vec_1.reshape(1,total_vocab_size)
        query = np.array(query)
        cosSim = cosine_similarity(doc, query)
        # list1 = cosSim[0].tolist()
        list1 = cosSim[0].item()
        cosine_score_dbl_nrml.append(list1)
    print(cosine_score_dbl_nrml)

    cosine_score_dbl_nrml_sorted = sorted(cosine_score_dbl_nrml)
    docName = []
    top5 = cosine_score_dbl_nrml_sorted[-5:]
    print(top5)
    for i in range(5):        
        print(top5[4-i]," and index is ",cosine_score_dbl_nrml.index(top5[4-i]))
        index = cosine_score_dbl_nrml.index(top5[4-i])
        head, tail = os.path.split(dataset[index][0])
        tail.strip()
        docName.append(tail.strip())
        docName.append(" with Cosine Similarity Value:")
        docName.append(top5[4-i])
        docName.append("\n")
    print("\nList of Top Documents wrt Cosine Similarity for i/p query(Double Normalization) is:")
    print(*docName)  
matching_score_5(5, "THE CRAB AND THE HERON")

Matching Score

Query: THE CRAB AND THE HERON

['crab', 'heron']

[122, 26, 270, 418, 169]
8.901579233214175
4.776156503341964
2.2296098113058496
2.210867165782778
2.2099247671613873
['crabhern.txt', 'aesop11.txt', 'long1-3.txt', 'timem.hac', 'fgoose.txt']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02557373919250654, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3808980579800617, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0